# Trainer API

![Alt text](image-5.png)

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"] , examples["sentence2"] , truncation= True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)
# We don't do any rename_columns or drop columns or .with_format method. Trainer API will do all of this automatically for us.

In [2]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)



from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir= "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01)


# We need model and training arguments to train our model.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer = tokenizer,
)

trainer.train()

# We know only training loss score.
# This is because we didin't specify any metric for the evaluation.

  0%|          | 0/1150 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4466, 'learning_rate': 1.1304347826086957e-05, 'epoch': 2.17}
{'loss': 0.1549, 'learning_rate': 2.6086956521739132e-06, 'epoch': 4.35}
{'train_runtime': 88.2658, 'train_samples_per_second': 207.781, 'train_steps_per_second': 13.029, 'train_loss': 0.271648298346478, 'epoch': 5.0}


TrainOutput(global_step=1150, training_loss=0.271648298346478, metrics={'train_runtime': 88.2658, 'train_samples_per_second': 207.781, 'train_steps_per_second': 13.029, 'train_loss': 0.271648298346478, 'epoch': 5.0})

In [6]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions)
print(predictions.predictions.shape, predictions.label_ids.shape)

  0%|          | 0/26 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-3.0971541 ,  3.6679256 ],
       [ 2.3900542 , -3.0286093 ],
       [ 2.1584663 , -2.8025389 ],
       [-3.1019952 ,  3.6807728 ],
       [ 2.2973018 , -3.030518  ],
       [-3.0260735 ,  3.619936  ],
       [-2.8688343 ,  3.5233548 ],
       [-2.9308448 ,  3.4574294 ],
       [-2.7995389 ,  3.334907  ],
       [-3.075251  ,  3.607215  ],
       [-3.0425384 ,  3.641816  ],
       [ 2.398232  , -3.0470524 ],
       [ 1.6444138 , -2.4403968 ],
       [-2.5943491 ,  3.1205575 ],
       [-3.0040689 ,  3.6347759 ],
       [ 2.0689294 , -2.6721249 ],
       [-3.062056  ,  3.6543078 ],
       [ 1.276446  , -2.0506577 ],
       [-3.0553515 ,  3.652121  ],
       [ 2.3300571 , -2.8536372 ],
       [ 2.1444476 , -2.7347136 ],
       [-2.796668  ,  3.3449898 ],
       [ 2.146821  , -2.6478872 ],
       [-3.0197098 ,  3.6397529 ],
       [-2.9703064 ,  3.5690658 ],
       [-2.6991014 ,  3.0299232 ],
       [-2.471028  ,  3.0239654 ],
       [-3.0372589 ,  3.64

In [9]:
import numpy as np
from datasets import load_metric

metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis = -1)
metric.compute(predictions= preds, references= predictions.label_ids)

{'accuracy': 0.8504901960784313, 'f1': 0.8960817717206134}

### To monitor the evaluation metrics during training

In [11]:
import numpy as np
from transformers import AutoModelForSequenceClassification
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



training_args = TrainingArguments("test-trainer" , evaluation_strategy= "epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset= tokenized_datasets["validation"],
    data_collator= data_collator,
    tokenizer = tokenizer,
    compute_metrics= compute_metrics
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.34251299500465393, 'eval_accuracy': 0.8602941176470589, 'eval_f1': 0.8950276243093922, 'eval_runtime': 0.6969, 'eval_samples_per_second': 585.411, 'eval_steps_per_second': 73.176, 'epoch': 1.0}
{'loss': 0.5152, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.698995053768158, 'eval_accuracy': 0.8259803921568627, 'eval_f1': 0.8852988691437803, 'eval_runtime': 0.6123, 'eval_samples_per_second': 666.302, 'eval_steps_per_second': 83.288, 'epoch': 2.0}
{'loss': 0.2894, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.6120919585227966, 'eval_accuracy': 0.8651960784313726, 'eval_f1': 0.9040139616055847, 'eval_runtime': 0.5778, 'eval_samples_per_second': 706.11, 'eval_steps_per_second': 88.264, 'epoch': 3.0}
{'train_runtime': 66.4394, 'train_samples_per_second': 165.625, 'train_steps_per_second': 20.726, 'train_loss': 0.3275650345768371, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3275650345768371, metrics={'train_runtime': 66.4394, 'train_samples_per_second': 165.625, 'train_steps_per_second': 20.726, 'train_loss': 0.3275650345768371, 'epoch': 3.0})

In [ ]:
# Done.